In [ ]:
# default_exp constructor

# model_constructor

> Create pytorch model.

In [ ]:
#hide
from nbdev.showdoc import *
from fastcore.test import *

In [ ]:
# export
import torch.nn as nn
import torch
from collections import OrderedDict
from model_constructor.layers import *

# Stem

In [ ]:
# export
class Stem(nn.Sequential):
    """Base stem"""
    def __init__(self, c_in=3,  stem_sizes=[], stem_out=64, 
            use_bn=False, pool=True, bn_1st=False,  stride_on=0, 
            conv_layer=ConvLayer, **kwargs): 
        self.sizes = [c_in] + stem_sizes + [stem_out]
        num_layers = len(self.sizes)-1
        stem = [(f"conv{i}", conv_layer(self.sizes[i], self.sizes[i+1], 
                stride=2 if i==stride_on else 1, act=True,
                bn_layer=not use_bn if i==num_layers-1 else True, **kwargs
                )) for i in range(num_layers)]
        if pool: stem += [('pool', nn.MaxPool2d(kernel_size=3, stride=2, padding=1))] 
        if use_bn: stem.append(('bn', nn.BatchNorm2d(stem_out)))
        super().__init__(OrderedDict(stem))
    def extra_repr(self):
            return f"sizes: {self.sizes}"

In [ ]:
stem = Stem()
stem

Stem(
  sizes: [3, 64]
  (conv0): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
)

In [ ]:
xb = torch.randn(64, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([64, 64, 32, 32])

In [ ]:
stem = Stem(use_bn=True)
stem

Stem(
  sizes: [3, 64]
  (conv0): ConvLayer(
    (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
xb = torch.randn(64, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([64, 64, 32, 32])

In [ ]:
stem = Stem(use_bn=True, stem_sizes=[32], stride_on=1)
stem

Stem(
  sizes: [3, 32, 64]
  (conv0): ConvLayer(
    (conv): Conv2d(3, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (bn): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (act_fn): ReLU(inplace=True)
  )
  (conv1): ConvLayer(
    (conv): Conv2d(32, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (act_fn): ReLU(inplace=True)
  )
  (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
)

In [ ]:
xb = torch.randn(64, 3, 128, 128)
y = stem(xb)
y.shape

torch.Size([64, 64, 32, 32])

In [ ]:
assert y.shape, torch.Size([64, 64, 32, 32])

# Body

## BasicBlock

In [ ]:
# export
class BasicBlock(nn.Module):
    """Basic block (simplified) as in pytorch resnet"""
    def __init__(self, ni, nf,  expansion=1, stride=1,
                 bn_1st=False, zero_bn=False, 
#                  groups=1, base_width=64, dilation=1, norm_layer=None
                conv_layer=ConvLayer, **kwargs):
        super().__init__()
        self.downsample = not ni==nf or stride==2
        self.conv = nn.Sequential(OrderedDict([
            ('conv_0', conv_layer(ni, nf, stride=stride, bn_1st=bn_1st, **kwargs)),
            ('conv_1', conv_layer(nf, nf, zero_bn=zero_bn, bn_1st=bn_1st, **kwargs))]))
        if self.downsample:
            self.downsample = conv_layer(ni, nf, ks=1, stride=stride, act=False, **kwargs)
        self.merge = Noop()
        self.act_conn = act_fn
                         
    def forward(self, x):
        identity = x
        out = self.conv(x)
        if self.downsample:
            identity = self.downsample(x)
        return self.act_conn(self.merge(out + identity))

In [ ]:
b_block = BasicBlock(64,64)
b_block

BasicBlock(
  (conv): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
b_block = BasicBlock(64,64, stride=2)
b_block

BasicBlock(
  (conv): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 64, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
xb = torch.randn(64, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([64, 64, 16, 16])

In [ ]:
b_block = BasicBlock(64,128, stride=2)
b_block

BasicBlock(
  (conv): Sequential(
    (conv_0): ConvLayer(
      (conv): Conv2d(64, 128, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (conv_1): ConvLayer(
      (conv): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (act_fn): ReLU(inplace=True)
      (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
  )
  (downsample): ConvLayer(
    (conv): Conv2d(64, 128, kernel_size=(1, 1), stride=(2, 2), bias=False)
    (bn): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  )
  (merge): Noop()
  (act_conn): ReLU(inplace=True)
)

In [ ]:
xb = torch.randn(64, 64, 32, 32)
y = b_block(xb)
y.shape

torch.Size([64, 128, 16, 16])

## Body constructor

In [ ]:
# class Body(nn.Sequential): 
#     '''Constructor for body'''
#     def __init__(self, block, 
#                  body_in=64, body_out=512, 
#                  layer_szs=[64,128,256,], blocks=[2,2,2,2],
#                  expansion=1, **kwargs):  
#         layer_szs = [body_in//expansion] + layer_szs + [body_out]
#         num_layers = len(layer_szs)-1
#         layers = [(f"layer_{i}", self._make_layer(block, layer_szs[i], layer_szs[i+1], blocks[i], 1 if i==0 else 2, **kwargs))
#                     for i in range(num_layers)]
#         super().__init__(OrderedDict(layers))
#     def _make_layer(self, block, ni, nf, blocks, stride, **kwargs):
#         return nn.Sequential(OrderedDict(
#             [(f'block_{i}', block(ni if i==0 else nf, nf, stride if i==0 else 1, **kwargs))
#               for i in range(blocks)]))
                    

In [ ]:
# export
class BasicLayer(nn.Sequential):
    '''Layer from blocks'''
    def __init__(self, block, blocks, ni, nf, expansion, stride, **kwargs):
        self.ni = ni
        self.nf = nf
        self.blocks = blocks
        self.expansion = expansion
        super().__init__(OrderedDict(
            [(f'block_{i}', block(ni if i==0 else nf, nf, expansion, 
                               stride if i==0 else 1, **kwargs))
              for i in range(blocks)]))
    def extra_repr(self):
        return f'from {self.ni*self.expansion} to {self.nf}, {self.blocks} blocks, expansion {self.expansion}.'                    

class Body(nn.Sequential): 
    '''Constructor for body'''
    def __init__(self, block, 
                 body_in=64, body_out=512,  
                 bodylayer=BasicLayer, expansion=1,
                 layer_szs=[64,128,256,], blocks=[2,2,2,2],
                 **kwargs):  
        layer_szs = [body_in//expansion] + layer_szs + [body_out]
        num_layers = len(layer_szs)-1
        layers = [(f"layer_{i}", bodylayer(block, blocks[i], 
                            layer_szs[i], layer_szs[i+1], expansion,
                            1 if i==0 else 2,  **kwargs))
                    for i in range(num_layers)]
        super().__init__(OrderedDict(layers))
    

In [ ]:
body = Body(BasicBlock)
body

Body(
  (layer_0): BasicLayer(
    from 64 to 64, 2 blocks, expansion 1.
    (block_0): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
        (conv_1): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        )
      )
      (merge): Noop()
      (act_conn): ReLU(inplace=True)
    )
    (block_1): BasicBlock(
      (conv): Sequential(
        (conv_0): ConvLayer(
          (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
          (act_fn): ReLU(inplace=True)
          (bn): BatchNorm2d(64, eps=1e-05, m

In [ ]:
bs = 16
xb = torch.randn(bs, 64, 32, 32)
y = body(xb)
y.shape

torch.Size([16, 512, 4, 4])

In [ ]:
assert y.shape==torch.Size([bs, 512, 4, 4])

# Head

In [ ]:
# export 
class Head(nn.Sequential):
    '''base head'''
    def __init__(self, ni, nf, **kwargs):
        super().__init__(OrderedDict(
            [('pool', nn.AdaptiveAvgPool2d((1, 1))),
             ('flat', Flatten()),
             ('fc',   nn.Linear(ni, nf)),
             ]))

In [ ]:
head = Head(512, 10)

In [ ]:
xb = torch.randn(64, 512, 4, 4)
y = head(xb)
y.shape

torch.Size([64, 10])

# class Net

In [ ]:
# export
def init_model(model, zero_bn=False):
    '''Init model'''
    for m in model.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')

In [ ]:
# export 
class Net(nn.Sequential):
    '''Constructor for model'''
    def __init__(self,  stem=Stem,
                 body=Body, block=BasicBlock,
                 layer_szs=[64,128,256,], blocks=[2,2,2,2],
                 head=Head, 
                 c_in=3,  num_classes=1000, 
                 body_in=64, body_out=512, expansion=1,
                 bn_1st=False,
                init_type='normal', **kwargs):
        # c_in = 3
        # block_szs  = [64,128,128,256,256,512]
        super().__init__(OrderedDict([
            ('stem', stem(c_in=c_in,stem_out=body_in, **kwargs)),
            ('body', body(block, body_in, body_out, 
                        layer_szs=layer_szs, blocks=blocks, expansion=expansion, **kwargs)),
            ('head', head(body_out*expansion, num_classes, **kwargs))
            ]))
        init_model(self)

In [ ]:
model = Net()

In [ ]:
model

Net(
  (stem): Stem(
    sizes: [3, 64]
    (conv0): ConvLayer(
      (conv): Conv2d(3, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act_fn): ReLU(inplace=True)
    )
    (pool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  )
  (body): Body(
    (layer_0): BasicLayer(
      from 64 to 64, 2 blocks, expansion 1.
      (block_0): BasicBlock(
        (conv): Sequential(
          (conv_0): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
          )
          (conv_1): ConvLayer(
            (conv): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
            (act_fn): ReLU(inplace=True)
            (bn): BatchNorm

In [ ]:
bs_test = 16
xb = torch.randn(bs_test, 3, 128, 128)
y = model(xb)
y.shape

torch.Size([16, 1000])

In [ ]:
assert y.shape == torch.Size([bs_test, 1000]), f"size expected {bs_test}, 1000"

# model_constructor
by ayasyrev

In [ ]:
# hide
from nbdev.export import *
notebook2script()

Converted 00_constructor.ipynb.
Converted 01_layers.ipynb.
Converted 02_resnet.ipynb.
Converted 03_xresnet.ipynb.
Converted index.ipynb.
